In [1]:
import numpy as np
import pandas as pd

## Data Generation

In [2]:
x1 = np.random.uniform(0,20,100)
x3 = np.random.uniform(10,20,100)
x2 = np.random.uniform(-15,10,100)
sigma12 = 15
constant = 2.89
Y = x3 + 4.5*x2 +x1 +7*x3 +constant +10*np.random.standard_normal(len(x1))

## Modelling  Stadard Form Joint Distribution of x1, x2, x3 and Y

In [3]:
data = np.stack((Y,x1,x2,x3),axis = -1)
mu = np.mean(data,axis = 0)
cov = np.cov(data,rowvar=False)
arr = np.random.multivariate_normal(mu,cov,(100))
cov2 = np.matrix(cov)

In [4]:
cov_r1 ={'Y':cov2[:,0],'x1':cov2[:,1],'x2':cov2[:,2], 'x3':cov2[:,3]}


In [5]:
print('Printing Covariance Matrix')
print('Y', cov_r1['Y'])
print('x1', cov_r1['x1'])
print('x2', cov_r1['x2'])
print('x3', cov_r1['x3'])


Printing Covariance Matrix
Y [[1577.04754203]
 [  -7.19789727]
 [ 246.30089589]
 [  55.61535059]]
x1 [[-7.19789727]
 [32.68272829]
 [-7.10584828]
 [-1.11880159]]
x2 [[246.30089589]
 [ -7.10584828]
 [ 61.82919298]
 [ -1.55347405]]
x3 [[55.61535059]
 [-1.11880159]
 [-1.55347405]
 [ 8.1277496 ]]


In [6]:
mu_r1 = {'mu_Y':mu[0],'mu_x1':mu[1],'mu_x2':mu[2],'mu_x3':mu[3]}

In [7]:
mu_r1

{'mu_Y': 125.97258268596148,
 'mu_x1': 10.539102165024804,
 'mu_x2': -1.70602816900933,
 'mu_x3': 15.053838162238357}

In [8]:
np.savetxt('mu_joined.txt',mu,delimiter=',')
np.savetxt('cov_joined.txt',cov,delimiter=',')
np.savetxt('joined_distribution.txt',arr,delimiter=',')

## Standard Form Conditional Distribution P(Y|x)

In [9]:
X_c = np.matrix(np.stack((x1,x2,x3),axis = -1))
Y = np.matrix(Y).reshape(np.shape(Y)[0],1)
mu_y = np.matrix(mu[0])
mu_x = np.transpose(np.matrix(mu[1:]))
s_yy = np.matrix(cov[0,0])
s_xx = np.matrix(cov[1:,1:])
s_xy = np.transpose(np.matrix(cov[1:,0]))
s_yx = np.transpose(s_xy)
beta1 = (s_xx**-1)*s_xy
beta0 = np.mean(Y) -np.transpose(beta1)*np.transpose(np.mean(X_c,axis =0))

error = s_yy-s_yx*(s_xx**-1)*s_xy
z = np.matrix(np.hstack((np.ones((100,1)),X_c)))
n = 1
z0 =np.matrix(np.hstack((np.ones((n,1)),X_c[0:n,:])))
Sigma_hat = z0*(np.transpose(z)*z)**-1*np.transpose(z0)
beta_hat = np.matrix(np.vstack((beta0,beta1)))
mu_hat = np.transpose(beta_hat)*np.transpose(z0)
mu_hat2 = np.array(mu_hat).reshape(n)
arr2 = np.random.multivariate_normal(mu_hat2,Sigma_hat,(100))


In [10]:
np.savetxt('mu_y_y_given_x.txt',mu_y,delimiter=',')
np.savetxt('mu_x_y_given_x.txt',mu_x,delimiter=',')
np.savetxt('Syy_y_given_x.txt',s_yy,delimiter=',')
np.savetxt('Sxx_y_given_x.txt',s_xx,delimiter=',')
np.savetxt('Sxy_y_given_x.txt',s_xy,delimiter=',')
np.savetxt('Syx_y_given_x.txt',s_yx,delimiter=',')

In [11]:
results = {'beta0':beta0,'beta1':beta1}
with open('results.txt', 'w') as f:
    for key in results.keys():
        f.write("%s,%s\n"%(key,results[key]))
np.savetxt('y_given_x.txt',arr2,delimiter = ',')

## Information Form, Joined Distribution

In [12]:
J = np.matrix(cov)**-1
data2 = np.matrix(data[0,:])
mu = np.transpose(np.matrix(mu))
h = J*mu
arg = np.dot(-0.5,data2*J*np.transpose(data2))+np.transpose(h)*np.transpose(data2)
Px = np.exp(arg)


In [13]:
print(J)

[[ 0.01072184 -0.01050622 -0.04601929 -0.08360774]
 [-0.01050622  0.04188254  0.04885141  0.08699268]
 [-0.04601929  0.04885141  0.21421847  0.36256246]
 [-0.08360774  0.08699268  0.36256246  0.7764059 ]]


In [14]:
Px

matrix([[4354510.67282293]])

In [15]:
J_r1 ={'Y':J[:,0],'x1':J[:,1],'x2':J[:,2], 'x3':J[:,3]} # J Matrix
h_r1 = {'h_Y':h[0],'h_x1':h[1],'h_x2':h[2],'h_x3':h[3]} # h Matrix

In [16]:
print('Printing J Matrix')
print('Y', J_r1['Y'])
print('x1', J_r1['x1'])
print('x2', J_r1['x2'])
print('x3', J_r1['x3'])


Printing J Matrix
Y [[ 0.01072184]
 [-0.01050622]
 [-0.04601929]
 [-0.08360774]]
x1 [[-0.01050622]
 [ 0.04188254]
 [ 0.04885141]
 [ 0.08699268]]
x2 [[-0.04601929]
 [ 0.04885141]
 [ 0.21421847]
 [ 0.36256246]]
x3 [[-0.08360774]
 [ 0.08699268]
 [ 0.36256246]
 [ 0.7764059 ]]


In [17]:
print('printing h matrix',h_r1)

printing h matrix {'h_Y': matrix([[0.05982398]]), 'h_x1': matrix([[0.34414058]]), 'h_x2': matrix([[-0.18982519]]), 'h_x3': matrix([[1.45388868]])}


In [18]:
np.savetxt('Information_Matrix_joined.txt',J,delimiter=',')
np.savetxt('h_matrix.txt',h,delimiter =',')
np.savetxt('joined_distribution.txt',Px,delimiter =',')

## Information Form Conditional Form Y|X

In [19]:
data_y =data2[0,0]
data_x = data2[:,1:]
J_yy = J[0,0]
J_xx = J[1:,1:]
J_xy = J[1:,0]
J_yx = J[0,1:]
hy = h[0]
hx = h[1:]
Arg_ygx = -0.5*(data_y*J_yy*data_y) + (hy-J_yx*np.transpose(data_x))*data_y
Py = np.exp(Arg_ygx)

In [20]:
Py

matrix([[2.53088951e+21]])

In [21]:
Conditional = {'J_yy':J_yy,'J_xx':J_xx,'J_xy':J_xy,'J_yx':J_yx,'hy':hy,'hx':hx,'input_x':data_x,'input_y':data_y, \
               'distribution':Py}
with open('Conditional_information.txt', 'w') as f:
    for key in Conditional.keys():
        f.write("%s,%s\n"%(key,Conditional[key]))

In [22]:
Conditional = {'J_yy':J_yy,'J_xx':J_xx,'J_xy':J_xy,'J_yx':J_yx,'hy':hy,'hx':hx,'input_x':data_x,'input_y':data_y, \
               'distribution':Py}
with open('Conditional_information.txt', 'w') as f:
    for key in Conditional.keys():
        f.write("%s,%s\n"%(key,Conditional[key]))

In [23]:
Py

matrix([[2.53088951e+21]])

In [24]:
"""
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
x = data[:,1:]
y =data[:,0]
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))
#mu = np.array([1, 2])
#cov = np.array([[.5, .25],[.25, .5]])
#rv = multivariate_normal(mu, cov)
Z = arr
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(Z[:,0].reshape(100,1),Z[:,1].reshape(100,1),Z[:,3].reshape(100,1))
fig.show()
"""

"\nfrom mpl_toolkits import mplot3d\nimport matplotlib.pyplot as plt\nfrom scipy.stats import multivariate_normal\nx = data[:,1:]\ny =data[:,0]\nX, Y = np.meshgrid(x, y)\npos = np.dstack((X, Y))\n#mu = np.array([1, 2])\n#cov = np.array([[.5, .25],[.25, .5]])\n#rv = multivariate_normal(mu, cov)\nZ = arr\nfig = plt.figure()\nax = fig.add_subplot(111, projection='3d')\nax.plot_surface(Z[:,0].reshape(100,1),Z[:,1].reshape(100,1),Z[:,3].reshape(100,1))\nfig.show()\n"

In [25]:
""""
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

x=data[:,2:3]
y = data[:,0].reshape(data.shape[0],1)
data2 = np.hstack((y,x))
sigma = np.cov(data2,rowvar=False)
mu_x = np.mean(x,axis =0)
mu_y = np.mean(y)
mu2 = np.hstack((mu_y,mu_x))
X, Y = np.meshgrid(x,y)


pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X; pos[:, :, 1] = Y
rv = multivariate_normal(mu2, sigma)

#Make a 3D plot
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, rv.pdf(pos),cmap='viridis',linewidth=0)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.show()
"""

'"\nimport numpy as np\nimport matplotlib.pyplot as plt\nfrom scipy.stats import multivariate_normal\nfrom mpl_toolkits.mplot3d import Axes3D\n\nx=data[:,2:3]\ny = data[:,0].reshape(data.shape[0],1)\ndata2 = np.hstack((y,x))\nsigma = np.cov(data2,rowvar=False)\nmu_x = np.mean(x,axis =0)\nmu_y = np.mean(y)\nmu2 = np.hstack((mu_y,mu_x))\nX, Y = np.meshgrid(x,y)\n\n\npos = np.empty(X.shape + (2,))\npos[:, :, 0] = X; pos[:, :, 1] = Y\nrv = multivariate_normal(mu2, sigma)\n\n#Make a 3D plot\nfig = plt.figure()\nax = fig.gca(projection=\'3d\')\nax.plot_surface(X, Y, rv.pdf(pos),cmap=\'viridis\',linewidth=0)\nax.set_xlabel(\'X axis\')\nax.set_ylabel(\'Y axis\')\nax.set_zlabel(\'Z axis\')\nplt.show()\n'

In [26]:
""""
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

#Parameters to set
mu_x = 0
variance_x = 3

mu_y = 0
variance_y = 15

#Create grid and multivariate normal
x = np.linspace(-10,10,500)
y = np.linspace(-10,10,500)
X, Y = np.meshgrid(x,y)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X; pos[:, :, 1] = Y
rv = multivariate_normal([mu_x, mu_y], [[variance_x, 0], [0, variance_y]])

#Make a 3D plot
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, rv.pdf(pos),cmap='viridis',linewidth=0)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.show()
"""

'"\nimport numpy as np\nimport matplotlib.pyplot as plt\nfrom scipy.stats import multivariate_normal\nfrom mpl_toolkits.mplot3d import Axes3D\n\n#Parameters to set\nmu_x = 0\nvariance_x = 3\n\nmu_y = 0\nvariance_y = 15\n\n#Create grid and multivariate normal\nx = np.linspace(-10,10,500)\ny = np.linspace(-10,10,500)\nX, Y = np.meshgrid(x,y)\npos = np.empty(X.shape + (2,))\npos[:, :, 0] = X; pos[:, :, 1] = Y\nrv = multivariate_normal([mu_x, mu_y], [[variance_x, 0], [0, variance_y]])\n\n#Make a 3D plot\nfig = plt.figure()\nax = fig.gca(projection=\'3d\')\nax.plot_surface(X, Y, rv.pdf(pos),cmap=\'viridis\',linewidth=0)\nax.set_xlabel(\'X axis\')\nax.set_ylabel(\'Y axis\')\nax.set_zlabel(\'Z axis\')\nplt.show()\n'